In [ ]:
import json
import chromadb
from sentence_transformers import SentenceTransformer

In [ ]:
JSON_PATRH = "data/prepared_dataset.json"
CHROMA_PATH = "/Users/mateuszbulanda-gorol/Desktop/Projects/rag_lex_project/vectordb/chroma_db"
COLLECTION_NAME = "isap_acts"
MODEL = "clarin-pl/roberta-polish-sentence-transformer-v1"
BATCH_SIZE = 64

In [ ]:
with open(JSON_PATRH, "r", encoding="utf-8") as f:
    dataset = json.load(f)

In [ ]:
model = SentenceTransformer(MODEL)
client = chromadb.PersistentClient(path=CHROMA_PATH)
collection = client.get_or_create_collection(name=COLLECTION_NAME)

In [ ]:
texts = [item["text"] for item in dataset]
ids = [f"{item['metadata']['filename']}_{item['metadata']['chunk_id']}" for item in dataset]
metadatas = [item["metadata"] for item in dataset]

In [ ]:
for i in range(len(texts), BATCH_SIZE):
    batch_texts = texts[i:i+BATCH_SIZE]
    batch_embeddings = model.encode(batch_texts, show_progress_bar=False).tolist()
    batch_ids = ids[i:i+BATCH_SIZE]
    batch_metadatas = metadatas[i:i+BATCH_SIZE]

    collection.add(
        ids=batch_ids
        ,documents=batch_texts
        ,metadatas=batch_metadatas
        ,embeddings=batch_embeddings
    )

print("✅ Baza wektorowa została utworzona i zapisana w:", CHROMA_PATH)

In [ ]:
##############################################################################

In [ ]:
# Zapytanie do bazy

from unittest import result


query = "Jakie są przepisy dotyczące ochrony środowiska?"
query_embedding = model.encode([query]).tolist()

results = collection.query(query_embeddings=query_embedding, n_results=3)

for doc, meta in zip(result["documents"][0], results["metadatas"][0]):
    print("----")
    print("Fragment:", doc[:300], "...")
    print("Metadane:", meta)